# Importing all necessary packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cap-4611-2021-fall-assignment-3/sample_submission.csv
/kaggle/input/cap-4611-2021-fall-assignment-3/eval.csv
/kaggle/input/cap-4611-2021-fall-assignment-3/train.csv


# Reading Data

In [2]:
dataframe = pd.read_csv('/kaggle/input/cap-4611-2021-fall-assignment-3/train.csv', low_memory=False) # Training Data
train_dataframe = pd.read_csv('/kaggle/input/cap-4611-2021-fall-assignment-3/eval.csv', low_memory=False) # Testing Data

# Exploratory Data Analysis

In [3]:
dataframe.head()

,id,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
0,0,73.516695,21.340907,18.686189,16.515660,13.986340,13.724160,13.546577,13.514781,13.483112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67498,-6.814462
1,1,388.023441,27.882166,20.729171,20.649087,20.449092,18.762317,18.311063,16.011055,15.832926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65270,-8.776741
2,2,73.516695,20.511719,20.511717,18.642329,18.640126,18.632230,18.630913,14.895390,14.894787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3022,-13.543936
3,3,53.358707,16.499291,16.424568,15.764738,15.324091,15.321921,13.798724,13.726431,13.707620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66725,-12.847512
4,4,53.358707,15.265913,15.191498,15.190635,13.687368,13.686528,13.653668,13.653627,13.653229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62436,-17.290178


In [4]:
dataframe.tail()

,id,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
12988,12988,73.516695,20.703867,14.234935,12.691563,12.610370,12.467563,12.436033,10.455215,8.991166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69817,-7.010101
12989,12989,53.358707,16.499601,16.498606,16.468071,16.466495,15.736080,15.674134,13.795118,13.791873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72565,-12.906913
12990,12990,73.516695,21.106224,20.673147,16.147199,16.141857,14.168558,14.163334,13.110054,12.777183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50259,-16.793042
12991,12991,73.516695,20.780679,18.702405,18.560051,17.883523,17.779027,17.739020,17.706468,14.623428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23340,-13.058719
12992,12992,36.858105,12.495330,12.495178,12.495178,12.494212,12.185325,12.184241,12.183496,12.182865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14020,-11.376928


In [5]:
dataframe.describe()

,id,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
count,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,...,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000,12993.000000
mean,6496.000000,116.244139,22.484154,20.507422,18.563918,17.199824,15.847108,15.150804,14.487618,13.974674,...,0.000150,0.000148,0.004156,0.000147,0.000144,0.002347,0.000142,0.001347,33101.068883,-11.179444
std,3750.900359,113.729597,8.699268,7.711998,6.484718,5.505691,4.176694,3.868422,3.516000,3.367758,...,0.002885,0.002859,0.045397,0.002830,0.002782,0.034180,0.002740,0.025917,23401.951232,3.669570
min,0.000000,36.858105,2.906146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-23.245373
25%,3248.000000,73.516695,17.993974,16.229604,15.176379,13.745581,13.653224,13.639635,12.762280,12.588049,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12309.000000,-13.475919
50%,6496.000000,73.516695,20.665957,18.632443,17.714561,16.021554,15.169079,13.877660,13.663518,13.652853,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27826.000000,-10.828420
75%,9744.000000,73.516695,21.139489,20.740126,18.714209,18.337209,17.667775,16.164715,15.497429,14.900133,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54864.000000,-8.616500
max,12992.000000,388.023441,66.779073,66.269180,66.268891,66.268756,66.268196,66.264158,66.258487,66.258177,...,0.062225,0.061999,0.500000,0.061534,0.059760,0.500000,0.057834,0.500000,74977.000000,-0.789513


# Feature Engineering

In [6]:
df = dataframe.drop(['id', 'pubchem_id'], axis=1)
df.shape

(12993, 1276)

In [7]:
X = df.drop(['Eat'], axis=1)
Y = df['Eat']
x = X.values
y = Y.values

# Splitting data into training and testing sets

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)
x_train.shape

(11693, 1275)

# Building Sequential Model

In [9]:
model = Sequential()
model.add(Dense(1000, input_dim=x.shape[1], kernel_initializer='normal', activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(500, kernel_initializer='normal', activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(50, kernel_initializer='normal'))
model.add(Dropout(0.25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
monitor = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)
model.summary()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/1000
366/366 - 5s - loss: 3.7766 - root_mean_squared_error: 1.9434 - val_loss: 0.4258 - val_root_mean_squared_error: 0.6525
Epoch 2/1000
366/366 - 3s - loss: 1.7383 - root_mean_squared_error: 1.3184 - val_loss: 0.3251 - val_root_mean_squared_error: 0.5701
Epoch 3/1000
366/366 - 3s - loss: 1.6210 - root_mean_squared_error: 1.2732 - val_loss: 0.1766 - val_root_mean_squared_error: 0.4202
Epoch 4/1000
366/366 - 3s - loss: 1.6157 - root_mean_squared_error: 1.2711 - val_loss: 0.3047 - val_root_mean_squared_error: 0.5520
Epoch 5/1000
366/366 - 3s - loss: 1.5903 - root_mean_squared_error: 1.2611 - val_loss: 0.1716 - val_root_mean_squared_error: 0.4143
Epoch 6/1000
366/366 - 3s - loss: 1.5468 - root_mean_squared_error: 1.2437 - val_loss: 0.1462 - val_root_mean_squared_error: 0.3824
Epoch 7/1000
366/366 - 3s - loss: 1.5102 - root_mean_squared_error: 1.2289 - val_loss: 0.4710 - val_root_mean_squared_error: 0.6863
Epoch 8/1000
366/366 - 3s - loss: 1.4807 - root_mean_squared_error: 1.2168 -

In [10]:
model1 = Sequential()
model1.add(Dense(1000, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(500, kernel_initializer='normal', activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(50, kernel_initializer='normal'))
model1.add(Dropout(0.25))
model1.add(Dense(1))
model1.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
monitor = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')
model1.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)
model1.summary()

Epoch 1/1000
366/366 - 4s - loss: 53.2277 - root_mean_squared_error: 7.2957 - val_loss: 6.9703 - val_root_mean_squared_error: 2.6401
Epoch 2/1000
366/366 - 3s - loss: 8.6892 - root_mean_squared_error: 2.9478 - val_loss: 5.3743 - val_root_mean_squared_error: 2.3182
Epoch 3/1000
366/366 - 3s - loss: 7.1875 - root_mean_squared_error: 2.6809 - val_loss: 3.6465 - val_root_mean_squared_error: 1.9096
Epoch 4/1000
366/366 - 3s - loss: 5.6402 - root_mean_squared_error: 2.3749 - val_loss: 7.8001 - val_root_mean_squared_error: 2.7929
Epoch 5/1000
366/366 - 3s - loss: 4.3867 - root_mean_squared_error: 2.0944 - val_loss: 3.9566 - val_root_mean_squared_error: 1.9891
Epoch 6/1000
366/366 - 3s - loss: 3.9792 - root_mean_squared_error: 1.9948 - val_loss: 4.2768 - val_root_mean_squared_error: 2.0680
Epoch 7/1000
366/366 - 3s - loss: 3.4714 - root_mean_squared_error: 1.8632 - val_loss: 6.0786 - val_root_mean_squared_error: 2.4655
Epoch 8/1000
366/366 - 3s - loss: 3.1039 - root_mean_squared_error: 1.7618 

In [11]:
model2 = Sequential()
model2.add(Dense(1000, input_dim=x.shape[1], kernel_initializer='normal', activation='selu'))
model2.add(Dropout(0.25))
model2.add(Dense(500, kernel_initializer='normal', activation='selu'))
model2.add(Dropout(0.25))
model2.add(Dense(50, kernel_initializer='normal'))
model2.add(Dropout(0.25))
model2.add(Dense(1))
model2.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
monitor = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')
model2.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)
model2.summary()

Epoch 1/1000
366/366 - 4s - loss: 102.6952 - root_mean_squared_error: 10.1339 - val_loss: 2.8712 - val_root_mean_squared_error: 1.6945
Epoch 2/1000
366/366 - 3s - loss: 7.2709 - root_mean_squared_error: 2.6965 - val_loss: 1.6703 - val_root_mean_squared_error: 1.2924
Epoch 3/1000
366/366 - 3s - loss: 5.1838 - root_mean_squared_error: 2.2768 - val_loss: 1.8522 - val_root_mean_squared_error: 1.3610
Epoch 4/1000
366/366 - 3s - loss: 4.7595 - root_mean_squared_error: 2.1816 - val_loss: 1.5839 - val_root_mean_squared_error: 1.2585
Epoch 5/1000
366/366 - 3s - loss: 4.3092 - root_mean_squared_error: 2.0759 - val_loss: 1.6777 - val_root_mean_squared_error: 1.2953
Epoch 6/1000
366/366 - 3s - loss: 4.0538 - root_mean_squared_error: 2.0134 - val_loss: 2.8206 - val_root_mean_squared_error: 1.6795
Epoch 7/1000
366/366 - 3s - loss: 3.7198 - root_mean_squared_error: 1.9287 - val_loss: 0.7885 - val_root_mean_squared_error: 0.8880
Epoch 8/1000
366/366 - 3s - loss: 3.5628 - root_mean_squared_error: 1.887

# Saving solutions to CSVs

In [12]:
solution = np.array(model2(train_dataframe.drop(['id', 'pubchem_id'], axis=1).to_numpy())).flatten()
output = pd.DataFrame({'id': train_dataframe['id'], 'Eat': solution})
display(output)
output.to_csv('submission.csv', index=False)

print('Your submission was successfully saved') # Asserting submission was successfully saved

,id,Eat
0,0,-10.297356
1,1,-12.839081
2,2,-16.750654
3,3,-12.256950
4,4,-8.675407
...,...,...
3244,3244,-10.366177
3245,3245,-11.822443
3246,3246,-9.254213
3247,3247,-8.171349


Your submission was successfully saved


# Best Model Selection

After submitting each of our three submission files, we found that our third model, using 'selu' activation, or Scaled Exponential Linear Unit activation produced the lowest and best score for this competition. 